<a href="https://colab.research.google.com/github/gomlfx/ApiaryFund/blob/master/random_forest_regression_2_eurusd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
# Import matplotlib for plotting and use magic command for Jupyter Notebooks
import matplotlib.pyplot as plt
%matplotlib inline
#import normalization
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
#using eurusd, change to your preferred asset
eu = pd.read_csv('EURUSD1440.csv')
eu.tail(2)

,1971.01.04,00:00,0.53690,0.53690.1,0.53690.2,0.53690.3,1
12761,2021.11.04,00:00,1.16085,1.16163,1.15281,1.15541,85558
12762,2021.11.05,00:00,1.15505,1.15509,1.15505,1.15508,5


In [3]:
eu.columns=['Date','Time','eu_Open','eu_High','eu_Low','eu_Close','Volume']
#Use D1 at least 2000 observations, change below for the number of rows you have.
#session crash or slow computing time may require you to use less rows. 
#eu = eu.iloc[-2000:] 
eu = eu.iloc[-8000:] 
eu = eu.reset_index(drop=True)
eu['Index']=eu.index
eu.set_index('Index', drop=True, append=False, inplace=True, verify_integrity=False)
#and make index a feature
eu.reset_index(inplace=True)
#drop last rows if needed
eu = eu[:-1]
print(eu.head())
print(eu.tail())

   Index        Date   Time  eu_Open  eu_High  eu_Low  eu_Close  Volume
0      0  1990.08.27  00:00   1.2647   1.2686  1.2539    1.2597    2441
1      1  1990.08.28  00:00   1.2598   1.2671  1.2547    1.2560    2101
2      2  1990.08.29  00:00   1.2567   1.2612  1.2546    1.2570    1291
3      3  1990.08.30  00:00   1.2494   1.2510  1.2454    1.2503    1031
4      4  1990.08.31  00:00   1.2503   1.2555  1.2387    1.2407    2401
      Index        Date   Time  eu_Open  eu_High   eu_Low  eu_Close  Volume
7994   7994  2021.10.29  00:00  1.16794  1.16899  1.15350   1.15622  100657
7995   7995  2021.11.01  00:00  1.15567  1.16088  1.15458   1.16064   71633
7996   7996  2021.11.02  00:00  1.16019  1.16132  1.15750   1.15790   74444
7997   7997  2021.11.03  00:00  1.15801  1.16161  1.15621   1.16123   89626
7998   7998  2021.11.04  00:00  1.16085  1.16163  1.15281   1.15541   85558


In [4]:
#for close prediction
#select rows to use for x,y
x = eu.iloc[:,[0]].values
print(x)
y = eu.iloc[:,[6]].values
print(y)

[[   0]
 [   1]
 [   2]
 ...
 [7996]
 [7997]
 [7998]]
[[1.2597 ]
 [1.256  ]
 [1.257  ]
 ...
 [1.1579 ]
 [1.16123]
 [1.15541]]


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [6]:
#for close prediction and accuracy test
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
# create regressor object
regressor = RandomForestRegressor(n_estimators = 10000, n_jobs=-1, criterion='mae', random_state = 1)
# fit the regressor with x and y data
regressor.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=2000, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [7]:
y_pred = regressor.predict(X_test)

In [9]:
def mean_absolute_percentage_error(y_true,y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test,y_pred)

In [11]:
#for regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)


MAE: 0.0049405975843747445
MSE: 4.6875967394385485e-05
RMSE: 0.006846602616946998
MAPE: 13.830524429615862


In [68]:
#for close prediction
y_pred = regressor.predict(np.array([7999]).reshape(1, 1)) # test the output by changing values
y_pred

array([1.16085312])

In [ ]:
# #for close prediction
# # Visualising the Random Forest Regression results

# # arange for creating a range of values
# # from min value of x to max
# # value of x with a difference of 0.01
# # between two consecutive values
# X_grid = np.arange(min(x), max(x), .01)

# # reshape for reshaping the data into a len(X_grid)*1 array,
# # i.e. to make a column out of the X_grid value				
# X_grid = X_grid.reshape((len(X_grid), 1))

# # Scatter plot for original data
# plt.scatter(x, y, color = 'blue')

# # plot predicted data
# plt.plot(X_grid, regressor.predict(X_grid),
# 		color = 'green')
# plt.title('Random Forest Regression')
# plt.xlabel('Position level')
# plt.ylabel('EURUSD')
# plt.xlim(80,100)
# plt.ylim(1.15,1.167)
# plt.show()


In [12]:
#for low prediction
#select rows to use for x,y
x = eu.iloc[:,[0]].values
print(x)
y = eu.iloc[:,[5]].values
print(y)

[[   0]
 [   1]
 [   2]
 ...
 [7996]
 [7997]
 [7998]]
[[1.2539 ]
 [1.2547 ]
 [1.2546 ]
 ...
 [1.1575 ]
 [1.15621]
 [1.15281]]


In [13]:
#for low prediction
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [28]:
#for low prediction and accuracy test
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
# create regressor object
regressor = RandomForestRegressor(n_estimators = 10000, n_jobs=-1, criterion='mae', random_state = 2)
# fit the regressor with x and y data
regressor.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=2000, n_jobs=-1, oob_score=False,
                      random_state=2, verbose=0, warm_start=False)

In [29]:
#for low pred 
y_pred = regressor.predict(X_test)
mape = mean_absolute_percentage_error(y_test,y_pred)

In [30]:
#for low pred regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)

MAE: 0.004395564028123851
MSE: 3.9120892924876805e-05
RMSE: 0.006254669689510135
MAPE: 14.06445250451888


In [74]:
#for low pred 
y_pred = regressor.predict(np.array([7999]).reshape(1, 1)) # test the output by changing values
y_pred

array([1.15377476])

In [ ]:
# #for low prediction
# # Visualising the Random Forest Regression results

# # arange for creating a range of values
# # from min value of x to max
# # value of x with a difference of 0.01
# # between two consecutive values
# X_grid = np.arange(min(x), max(x), .01)

# # reshape for reshaping the data into a len(X_grid)*1 array,
# # i.e. to make a column out of the X_grid value				
# X_grid = X_grid.reshape((len(X_grid), 1))

# # Scatter plot for original data
# plt.scatter(x, y, color = 'blue')

# # plot predicted data
# plt.plot(X_grid, regressor.predict(X_grid),
# 		color = 'green')
# plt.title('Random Forest Regression')
# plt.xlabel('Position level')
# plt.ylabel('EURUSD')
# plt.xlim(80,100)
# plt.ylim(1.15,1.167)
# plt.show()

In [31]:
#for high prediction
#select rows to use for x,y
x = eu.iloc[:,[0]].values
print(x)
y = eu.iloc[:,[4]].values
print(y)

[[   0]
 [   1]
 [   2]
 ...
 [7996]
 [7997]
 [7998]]
[[1.2686 ]
 [1.2671 ]
 [1.2612 ]
 ...
 [1.16132]
 [1.16161]
 [1.16163]]


In [32]:
#for high prediction
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [33]:
#for high prediction and accuracy test
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
# create regressor object
regressor = RandomForestRegressor(n_estimators = 10000, n_jobs=-1, criterion='mae', random_state = 3)
# fit the regressor with x and y data
regressor.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10000, n_jobs=-1, oob_score=False,
                      random_state=3, verbose=0, warm_start=False)

In [34]:
#for high prediction
y_pred = regressor.predict(X_test)
mape = mean_absolute_percentage_error(y_test,y_pred)

In [35]:
#for high regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)

MAE: 0.004358945131874337
MSE: 3.8017134852548364e-05
RMSE: 0.0061658036663964876
MAPE: 13.882898113992484


In [36]:
#for high prediction
y_pred = regressor.predict(np.array([7999]).reshape(1, 1)) # test the output by changing values
y_pred

array([1.16170369])

In [ ]:
# #for high prediction
# # Visualising the Random Forest Regression results

# # arange for creating a range of values
# # from min value of x to max
# # value of x with a difference of 0.01
# # between two consecutive values
# X_grid = np.arange(min(x), max(x), .01)

# # reshape for reshaping the data into a len(X_grid)*1 array,
# # i.e. to make a column out of the X_grid value				
# X_grid = X_grid.reshape((len(X_grid), 1))

# # Scatter plot for original data
# plt.scatter(x, y, color = 'blue')

# # plot predicted data
# plt.plot(X_grid, regressor.predict(X_grid),
# 		color = 'green')
# plt.title('Random Forest Regression')
# plt.xlabel('Position level')
# plt.ylabel('EURUSD')
# plt.xlim(7900,8000)
# plt.ylim(1.155,1.19)
# plt.show()